In [1]:
import cv2 
from scipy.spatial import distance as dist
import playsound
import numpy as np
import dlib
import imutils
from imutils import face_utils
from pygame import mixer
import argparse
from imutils.video import VideoStream
import time

###***** Uncomment for RasPi *****###

#from picamera2 import Picamera2


"""
import time
import face_recognition
from threading import Thread
tutorial link : https://youtu.be/u7_34o_rs8o?si=dbzFq0jbhUL2SVCG
"""

pygame 2.6.1 (SDL 2.28.4, Python 3.11.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


'\nimport time\nimport face_recognition\nfrom threading import Thread\ntutorial link : https://youtu.be/u7_34o_rs8o?si=dbzFq0jbhUL2SVCG\n'

In [2]:
# Caculating the eye aspect ratio
def EAR(eye):
    #Eye points coordinates X,Y
    A = dist.euclidean(eye[1] ,eye[5])
    B = dist.euclidean(eye[2] ,eye[4])
    C = dist.euclidean(eye[0] ,eye[3])
    ear = (A+B)/(2*C)
    return ear

In [3]:
threshold = 0.25
frame_check = 20
detect = dlib.get_frontal_face_detector() #more faster than the cascaded method & uses svm
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [4]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["right_eye"]
# Option 0
cap = cv2.VideoCapture( 0 ) #to select the first camera if multiple cameras exist

# OPTION 1
# ap = argparse.ArgumentParser()
# ap.add_argument("-w", "--webcam", type=int, default=0,
#                 help="index of webcam on system")
# args = vars(ap.parse_args())
# vs = VideoStream(src=args["webcam"]).start() 

# Option 2
# vs= VideoStream(usePiCamera=True).start()

# Option 3
# camera = Picamera2()
# camera_config = camera.create_preview_configuration(main={"size": (640, 480)})
# camera.configure(camera_config)
# camera.start()

# Option 4
# camera = Picamera2()
# camera.preview_configuration.main.size = (800,800)
# camera.preview_configuration.main.format = "RGB888"
# camera.preview_configuration.align()
# camera.configure("preview")
# camera.start()

flag = 0


In [5]:
#mixer Should be initialized to work
mixer.init()
mixer.music.load("Untitled (1).mp3")

In [6]:
while True:
    ret, frame=cap.read()
    if not ret:
        print("Frame not read")
        break
    
    
    # For both option 1 and 2 for Ras_PI
    # frame = vs.read()
    
    #For Option 3 and 4
    #frame= camera.capture_array()

    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    subjects = detect(gray , 0)
    
    for subject in subjects:
        shape = predict(gray,subject)
        shape = face_utils.shape_to_np(shape)
        leftEye = shape[lStart: lEnd]
        rightEye = shape[rStart: rEnd]
        
        #calculating Eye Aspect Ratio
        leftEAR = EAR(leftEye)
        rightEAR = EAR(rightEye)
        ear = (leftEAR + rightEAR) / 2.0
        
        #these next 4 lines for drawing a green line around the eye
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        
        #Drowsiness detection
        if ear <threshold :
            flag +=1
            print(flag)
            if flag >= frame_check:
                cv2.putText(frame, "************alert********" , 
                (10 , 30 ) ,cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255) , 2)
                
                cv2.putText(frame, "****************ALERT!****************", (10,325),
				cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                mixer.music.play()
                time.sleep(3)
        else: 
            flag = 0
    cv2.imshow("Frame" , frame)
    cv2.waitKey(1) #display every 1 mS1
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
    
cv2.destroyAllWindows()
cap.release()

1
2
3
4
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2


KeyboardInterrupt: 

pygame 2.6.1 (SDL 2.28.4, Python 3.11.13)
Hello from the pygame community. https://www.pygame.org/contribute.html
Pygame audio failed: ndarray is not C-contiguous
Falling back to system beep
Driver Drowsiness Detection System Started
Press 'q' to quit
Make sure the camera can see your face clearly


AttributeError: 'DrowsinessDetector' object has no attribute 'alert_sound'

In [2]:
"""
Driver Drowsiness and Yawning Detection System

Requirements:
1. Install packages: pip install opencv-python dlib numpy scipy pygame
2. Download shape_predictor_68_face_landmarks.dat from http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
3. Place your MP3 alert file "Untitled (1).mp3" in the same directory
4. Ensure webcam is connected and working

Usage:
- Run the script and position your face clearly in front of the camera
- The system will play "Untitled (1).mp3" when drowsiness or yawning is detected
- Press 'q' to quit
"""

import cv2
import dlib
import numpy as np
from scipy.spatial import distance as dist
import pygame
import time
import sys

class DrowsinessDetector:
    def __init__(self, use_pygame_audio=True, mp3_filename="Untitled (1).mp3"):
        # Initialize dlib's face detector and facial landmark predictor
        self.detector = dlib.get_frontal_face_detector()
        self.predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
        
        # Eye and mouth landmark indices
        self.LEFT_EYE = [36, 37, 38, 39, 40, 41]
        self.RIGHT_EYE = [42, 43, 44, 45, 46, 47]
        self.MOUTH = [48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
        
        # Thresholds
        self.EYE_AR_THRESH = 0.25  # Eye aspect ratio threshold for drowsiness
        self.MOUTH_AR_THRESH = 0.7  # Mouth aspect ratio threshold for yawning
        self.EYE_AR_CONSEC_FRAMES = 20  # Consecutive frames for drowsiness detection
        self.YAWN_CONSEC_FRAMES = 15   # Consecutive frames for yawn detection
        
        # Counters
        self.eye_counter = 0
        self.yawn_counter = 0
        self.drowsy_alert = False
        self.yawn_alert = False
        
        # Audio setup
        self.use_pygame_audio = use_pygame_audio
        self.mp3_file = mp3_filename
        if self.use_pygame_audio:
            try:
                pygame.mixer.init()
                self.load_mp3_sound()
                print(f"Audio alerts enabled (pygame) - Using custom MP3: {self.mp3_file}")
            except Exception as e:
                print(f"Pygame audio failed: {e}")
                print("Falling back to system beep")
                self.use_pygame_audio = False
        else:
            print("Using system beep for alerts")
    
    def load_mp3_sound(self):
        """Load the MP3 alert sound"""
        try:
            self.alert_sound = pygame.mixer.Sound(self.mp3_file)
            print(f"Successfully loaded: {self.mp3_file}")
        except pygame.error as e:
            print(f"Error loading MP3 file '{self.mp3_file}': {e}")
            print("Make sure the file is in the same directory as this script")
            raise
    
    def play_alert(self):
        """Play alert sound"""
        if self.use_pygame_audio:
            self.alert_sound.play()
        else:
            # Use system beep as fallback
            print('\a')  # ASCII bell character
            sys.stdout.flush()
    
    def eye_aspect_ratio(self, eye):
        """Calculate the eye aspect ratio (EAR)"""
        # Vertical distances
        A = dist.euclidean(eye[1], eye[5])
        B = dist.euclidean(eye[2], eye[4])
        
        # Horizontal distance
        C = dist.euclidean(eye[0], eye[3])
        
        # EAR formula
        ear = (A + B) / (2.0 * C)
        return ear
    
    def mouth_aspect_ratio(self, mouth):
        """Calculate the mouth aspect ratio (MAR)"""
        # Vertical distances
        A = dist.euclidean(mouth[2], mouth[10])  # 51, 59
        B = dist.euclidean(mouth[4], mouth[8])   # 53, 57
        
        # Horizontal distance
        C = dist.euclidean(mouth[0], mouth[6])   # 49, 55
        
        # MAR formula
        mar = (A + B) / (2.0 * C)
        return mar
    
    def extract_landmarks(self, gray, rect):
        """Extract facial landmarks"""
        shape = self.predictor(gray, rect)
        coords = np.zeros((68, 2), dtype="int")
        
        for i in range(0, 68):
            coords[i] = (shape.part(i).x, shape.part(i).y)
        
        return coords
    
    def detect_drowsiness_yawning(self, frame):
        """Main detection function"""
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.detector(gray, 0)
        
        for face in faces:
            landmarks = self.extract_landmarks(gray, face)
            
            # Extract eye and mouth coordinates
            left_eye = landmarks[self.LEFT_EYE]
            right_eye = landmarks[self.RIGHT_EYE]
            mouth = landmarks[self.MOUTH]
            
            # Calculate aspect ratios
            left_ear = self.eye_aspect_ratio(left_eye)
            right_ear = self.eye_aspect_ratio(right_eye)
            ear = (left_ear + right_ear) / 2.0
            
            mar = self.mouth_aspect_ratio(mouth)
            
            # Draw eye and mouth contours
            cv2.drawContours(frame, [cv2.convexHull(left_eye)], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [cv2.convexHull(right_eye)], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [cv2.convexHull(mouth)], -1, (0, 255, 0), 1)
            
            # Drowsiness detection
            if ear < self.EYE_AR_THRESH:
                self.eye_counter += 1
                if self.eye_counter >= self.EYE_AR_CONSEC_FRAMES:
                    if not self.drowsy_alert:
                        self.drowsy_alert = True
                        self.play_alert()
            else:
                self.eye_counter = 0
                self.drowsy_alert = False
            
            # Yawning detection
            if mar > self.MOUTH_AR_THRESH:
                self.yawn_counter += 1
                if self.yawn_counter >= self.YAWN_CONSEC_FRAMES:
                    if not self.yawn_alert:
                        self.yawn_alert = True
                        self.play_alert()
            else:
                self.yawn_counter = 0
                self.yawn_alert = False
            
            # Display information
            cv2.putText(frame, f"EAR: {ear:.2f}", (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"MAR: {mar:.2f}", (10, 60),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            # Alert messages
            if self.drowsy_alert:
                cv2.putText(frame, "DROWSINESS ALERT!", (10, 100),
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 0, 255), 10)
            
            if self.yawn_alert:
                cv2.putText(frame, "YAWNING DETECTED!", (10, 150),
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 165, 255), 3)
        
        return frame

def main():
    # Initialize the detector
    detector = DrowsinessDetector()
    
    # Initialize webcam
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    print("Driver Drowsiness Detection System Started")
    print(f"Alert sound: {detector.mp3_file}")
    print("Press 'q' to quit")
    print("Make sure the camera can see your face clearly")
    print(f"Ensure '{detector.mp3_file}' is in the same directory as this script")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Flip frame horizontally for mirror effect
        frame = cv2.flip(frame, 1)
        
        # Detect drowsiness and yawning
        frame = detector.detect_drowsiness_yawning(frame)
        
        # Display instructions
        cv2.putText(frame, "Press 'q' to quit", (10, frame.shape[0] - 20),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        # Show frame
        cv2.imshow("Driver Drowsiness Detection", frame)
        
        # Check for quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Cleanup
    cap.release()
    cv2.destroyAllWindows()
    if detector.use_pygame_audio:
        pygame.mixer.quit()

if __name__ == "__main__":
    main()

Successfully loaded: Untitled (1).mp3
Audio alerts enabled (pygame) - Using custom MP3: Untitled (1).mp3
Driver Drowsiness Detection System Started
Alert sound: Untitled (1).mp3
Press 'q' to quit
Make sure the camera can see your face clearly
Ensure 'Untitled (1).mp3' is in the same directory as this script
